![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Arts/ImageEditing/image-editing.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Image Editing

We can edit images using Python code and the [Pillow](https://python-pillow.org) library.

To start, let's open an image from a URL. For this we'll need the [Requests](https://requests.readthedocs.io/en/master) library for downloading and the [shutil](https://docs.python.org/3/library/shutil.html) module for saving the file. Then we can open and display the image.

In [ ]:
image_url = 'https://publicdomainvectors.org/photos/Alberta-Icon.png'

from PIL import Image, ImageFilter, ImageChops # the Pillow library
import requests                                # for downloading images
import shutil                                  # for saving downloaded images
from IPython.display import Image as IMG       # since Pillow doesn't display animated GIFs
import plotly.express as px                    # for plotting histograms

source = requests.get(image_url, stream=True).raw
destination = open('image.png', 'wb')
shutil.copyfileobj(source, destination)
image = Image.open('image.png')
image

## Image Attributes

To display image attributes, there are methods such as `format`, `mode`, `info`, and `size`.

In [ ]:
image.format

In [ ]:
image.mode

In [ ]:
image.info

In [ ]:
image.size

## Cropping an Image

To crop an image, we specify the starting `x` and `y` pixel coordinates at the top left and the ending `x` and `y` coordinates at the bottom right. Notice that we need a double set of `()`.

In [ ]:
image.crop((20,30,480,110))

## Transposing an Image

We can use the following `transpose` methods:
```
Image.FLIP_LEFT_RIGHT
Image.FLIP_TOP_BOTTOM
Image.ROTATE_90
Image.ROTATE_180
Image.ROTATE_270
Image.TRANSPOSE
Image.TRANSVERSE
```

In [ ]:
image.transpose(Image.FLIP_LEFT_RIGHT)

We can also [rotate](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.rotate) an image any number of degrees.

In [ ]:
image.rotate(angle=25)

## Converting to Grayscale

We can convert an image to grayscale using `convert('L')` where `L` means luma (brightness).

In [ ]:
image.convert('L')

## Image Filters

There are some image filters we can use:

```
BLUR
CONTOUR
DETAIL
EDGE_ENHANCE
EDGE_ENHANCE_MORE
EMBOSS
FIND_EDGES
SHARPEN
SMOOTH
SMOOTH_MORE
```

In [ ]:
image.filter(ImageFilter.EMBOSS)

We can try converting to grayscale first and then applying the filter.

In [ ]:
image.convert('L').filter(ImageFilter.EMBOSS)

## Outlining an Image

### Outlining with Grayscale

Edge detection using `FIND_EDGES` sometimes works best if we convert to grayscale first with `.convert('L')`

In [ ]:
image.convert('L').filter(ImageFilter.FIND_EDGES)

### Outlining with Split

You may find it works better to `FIND_EDGES` on the color image and then `split` by color band. The follow code will display each of the color band results. In our case there are four bands, since the image mode is `RGBA`.

In the previous examples we just used the `.` to string together multiple operations, this time we are going to create variable (`edges` and `bands`) to store our intermediate steps.

In [ ]:
edges = image.filter(ImageFilter.FIND_EDGES)
bands = edges.split()
for band in bands:
    display(band)

## Inverting Colors

We can use `ImageChops.invert()` to invert the colors of an image.

This doesn't work with images that have an [alpha matte](https://en.wikipedia.org/wiki/Alpha_compositing) layer, so we'll need to `convert` our example image from `RGBA` to `RGB`.

In [ ]:
image_rgb = image.convert('RGB')
ImageChops.invert(image_rgb)

Inverting an edge-detected greyscale image might be a good way to generate a coloring page.

In [ ]:
edges = image.convert('L').filter(ImageFilter.FIND_EDGES)
ImageChops.invert(edges)

## Creating Animated GIFs

We can create animated GIFs using some of these transformations.

In [ ]:
# Rotating
frames = []
for angle in range(36):
    frames.append(image.rotate(angle*10))
frames[0].save('rotating.gif',format='GIF',append_images=frames[1:],save_all=True,loop=0)
IMG(filename='rotating.gif')

In [ ]:
# Flipping
frames = [
    image,
    image.transpose(Image.FLIP_LEFT_RIGHT),
    image.transpose(Image.FLIP_TOP_BOTTOM),
    image.transpose(Image.FLIP_TOP_BOTTOM).transpose(Image.FLIP_LEFT_RIGHT),
    image.transpose(Image.TRANSPOSE),
    image.transpose(Image.TRANSVERSE),
]
frames[0].save('flipping.gif',format='GIF',append_images=frames[1:],duration=400,save_all=True,loop=0)
IMG(filename='flipping.gif')

In [ ]:
# Flashing inverted colors
frames = [image, ImageChops.invert(image.convert('RGB'))]
frames[0].save('flashing.gif',format='GIF',append_images=frames[1:],duration=200,save_all=True,loop=0)
IMG(filename='flashing.gif')

In [ ]:
# Cycling through filters
frames = [image,
          image.filter(ImageFilter.BLUR),
          image.filter(ImageFilter.CONTOUR),
          image.filter(ImageFilter.DETAIL),
          image.filter(ImageFilter.EDGE_ENHANCE),
          image.filter(ImageFilter.EDGE_ENHANCE_MORE),
          image.filter(ImageFilter.EMBOSS),
          image.filter(ImageFilter.FIND_EDGES),
          image.filter(ImageFilter.SHARPEN),
          image.filter(ImageFilter.SMOOTH),
          image.filter(ImageFilter.SMOOTH_MORE),
]
frames[0].save('flashing.gif',format='GIF',append_images=frames[1:],save_all=True,duration=200,loop=0)
IMG(filename='flashing.gif')

## Histograms

We can plot an [image histogram](https://en.wikipedia.org/wiki/Image_histogram) using the [Plotly Express](https://plotly.com/python/plotly-express) library.

For our simple example image this won't be very interesting though, so we'll look at some other histograms next.

In [ ]:
px.histogram(image.histogram())

For something more interesting, let's try plotting a histogram of a famous painting, [The Last Supper](https://en.wikipedia.org/wiki/The_Last_Supper_(Leonardo)).

In [ ]:
painting_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/The_Last_Supper_-_Leonardo_Da_Vinci_-_High_Resolution_32x16.jpg/1024px-The_Last_Supper_-_Leonardo_Da_Vinci_-_High_Resolution_32x16.jpg'
shutil.copyfileobj(requests.get(painting_url, stream=True).raw, open('painting.png', 'wb'))
painting = Image.open('painting.png')
painting

In [ ]:
px.histogram(painting.histogram(), title='Histogram of The Last Supper')

We can also look at the histogram for each channel of the image.

In [ ]:
for channel in painting.split():
    px.histogram(channel.histogram()).show()

If we want to add titles to the histograms and hide the legend we can run this code.

In [ ]:
for i in range(len(painting.split())):
    px.histogram(painting.split()[i].histogram(),
                 title='The Last Supper '+painting.mode[i]).update_layout(showlegend=False).show()

Here is a code cell you can use to download any image from a URL and display the histogram for each channel.

In [ ]:
img_url = 'https://pixnio.com/free-images/2017/05/31/2017-05-31-10-48-16-900x600.jpg'

shutil.copyfileobj(requests.get(img_url,stream=True).raw, open('img.png','wb'))
img = Image.open('img.png')
display(img)
for i in range(len(img.split())):
    px.histogram(img.split()[i].histogram(),title=painting.mode[i]+' Channel').update_layout(showlegend=False).show()

## Removing Image Files

If you'd like to remove (`!rm`) the image files created by this notebook, delete the `#` signs and run the following code cell.

In [ ]:
#!rm *.png
#!rm *.gif
print('Files in this directory:')
!ls

# Conclusion

There are other methods for working with images in Python and Jupyter, such as [matplotlib](https://matplotlib.org/3.2.1/tutorials/introductory/images.html), but hopefully this served as a good introduction to some useful parts of the [Pillow](https://python-pillow.org) library.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)